In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape
from keras.layers.merge import concatenate, multiply,add
import tensorflow as tf
from keras import regularizers
from keras.initializers import glorot_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy


import rpy2
import rpy2.robjects.numpy2ri
from stldecompose import decompose
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
stats = importr('stats')
stl=stats.stl
ts =stats.ts

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data= pd.read_csv("Data/FromMohammed/cif_dataset_complete.csv",header=None)
predictions = pd.read_csv("Data/FromMohammed/theta_25_horg.csv",index_col=0,skiprows = [1])
print(data.shape)
print(predictions.shape)

(72, 123)
(72, 90)


In [3]:
def make_input(data,window_size,horizon=1):
    length=data.shape[0]
#     depth=data.shape[2]
    y = np.zeros([length-window_size+1-horizon,horizon])
    output=np.zeros([length-window_size+1-horizon,window_size])
    for i in range(length-window_size+1-horizon):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def make_k_input(data,window_size,horizon):
    length = data.shape[0]
    output= np.zeros([length-window_size+1-horizon,horizon])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i+window_size:i+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
#     print(str(extra))
    data = np.append(data,np.zeros([horizon-extra]))
#     print(data)
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size,1), y
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def nonov_make_k_input(data,window_size,horizon):
    length = data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],extra)
    data = np.append(data,data_app)    
#     data = np.append(data,np.zeros([horizon-extra]))
    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
    return output.reshape(output.shape[0],horizon,1)    

In [4]:
def smape(val,preds,horizon):
    temp = np.abs(val-preds)
    temp1=np.abs(val)+np.abs(preds)
    
    smape = 200/horizon*np.sum(temp/temp1)

    return smape




def smape_nzr(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 200.0 * np.mean(diff)

# Auto-Encoder - All Theta rows

In [ ]:
theta_length = predictions.shape[0]

horizon=6
window_size=6
theta_all=np.zeros(0)
for i in range(theta_length):
    current_pred= np.asarray(predictions.iloc[i].dropna().values,dtype=float)
    theta_all=np.concatenate((theta_all,current_pred),axis=0)
theta_input = make_k_input(theta_all,window_size,horizon)
normalized_theta_input=np.zeros(theta_input.shape)
for j in range(theta_input.shape[0]):
    scaler = MinMaxScaler()
    scaler.fit(theta_input[j])
    normalized_theta_input[j]= scaler.transform(theta_input[j])
#######################################################################################
norm_reshaped_input=normalized_theta_input.reshape(normalized_theta_input.shape[0],6)
ncol = normalized_theta_input.shape[1] #6

encoding_dim = 4
input_dim = Input(shape = (ncol, ))
# Encoder Layers
encoded1 = Dense(5, activation = 'relu')(input_dim)
encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)
# Decoder Layers
decoded1 = Dense(5, activation = 'relu')(encoded2)
decoded2 = Dense(ncol, activation = 'sigmoid')(decoded1)
# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded2)
autoencoder.compile(optimizer = 'adam', loss = 'mse')

autoencoder.fit(norm_reshaped_input, norm_reshaped_input, epochs = 500, batch_size = 32, shuffle = False)
encoder = Model(inputs = input_dim, outputs = encoded2)
encoded_input = Input(shape = (encoding_dim, ))
auto_out = np.array(encoder.predict(norm_reshaped_input))

new_theta_input=auto_out.reshape(auto_out.shape[0],auto_out.shape[1],1)
autoencoder.save('MyModelCheckpoint/autoencoder_model.h5')
encoder.save('MyModelCheckpoint/encoder_model.h5')
np.savetxt('MyModelCheckpoint/new_theta_input.csv',auto_out, fmt='%1.3f',delimiter=',')

    

# Case1: Using only Data

In [21]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape1=np.zeros(data_length)
    for y in range(data_length):
####################################################################################
##############creating train, validation and test sets from Data####################
####################################################################################
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
    
        train = series_data[:-n_val]
        test = series_d[-(horizon+window_size):]
        val = series_data[-(2*n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)
        
        
        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            scaler1.fit(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1))
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            scaler2.fit(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1))
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            scaler3.fit(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1))
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]

        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]


####################################################################################
################    Creating inputs for Neural Network      ########################
####################################################################################
        #train_input = np.append(x_train,train_pred,axis=1)
        train_input = x_train
        #val_input = np.append(x_val,val_pred,axis=1)
        val_input = x_val
        #test_input = np.append(x_test,test_pred,axis=1)
        test_input = x_test

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        #input_data= Input(batch_shape=(None,window_size+horizon,1),name='input_data')
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

        model=Model(inputs=[input_data],outputs=net)
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF01.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

        model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)
        model.load_weights('MyModelCheckpoint/CIF01.h5')
        pred=model.predict({'input_data':test_input})
        pred=scaler3.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]
        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape1[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)
    
np.savetxt('MyModelCheckpoint/prediction01.csv',final_predictions, fmt='%1.3f',delimiter=',')

print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
print('-----------------')

2019-08-06 17:56:38.221055


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [52:49<00:00, 47.26s/it]


2019-08-06 18:49:27.265232
-----------------
SMAPE1: (DeepEX approach)
13.374298976722933
SMAPE2 (NZR approach)
13.374298976722933
MSE1:
2883138816411.7314
-----------------


# Case 2 , 3 , 4 , 5 : Using Autoencoder on Expert Knowledge

# Preparing Autoencoder for Expert Knowledge

In [ ]:
auto_out=pd.read_csv("MyModelCheckpoint/new_theta_input.csv",header=None)
encoding_dim2 = 32
input_dim2 = Input(shape = (4, ))
# Encoder Layers
encoded12 = Dense(8, activation = 'relu')(input_dim2)
encoded22 = Dense(16, activation = 'relu')(encoded12)
encoded32 = Dense(32, activation = 'relu')(encoded22)
# Decoder Layers
decoded12 = Dense(16, activation = 'relu')(encoded32)
decoded22 = Dense(8, activation = 'relu')(decoded12)
decoded32 = Dense(4, activation = 'sigmoid')(decoded22)

# Combine Encoder and Deocder layers
autoencoder2 = Model(inputs = input_dim2, outputs = decoded32)
autoencoder2.compile(optimizer = 'adam', loss = 'mse')
autoencoder2.fit(auto_out, auto_out, epochs = 100, batch_size = 32, shuffle = False)

encoder2 = Model(inputs = input_dim2, outputs = encoded32)
encoded_input2 = Input(shape = (encoding_dim2, ))


# Case 2: Adding Expert Knowledge to layer 1

In [ ]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape2=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
    
        train = series_data[:-n_val]
        test = series_d[-(horizon+window_size):]
        val = series_data[-(2*n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            scaler1.fit(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1))
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            scaler2.fit(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1))
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            scaler3.fit(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1))
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        #val_input = np.append(x_val,val_pred_auto,axis=1)
        val_input = x_val
        #test_input = np.append(x_test,test_pred_auto,axis=1)
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        train_p = series_pred[:-n_val]
        val_p = series_pred[-(2*n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l]=train_pred[l][-6:]
            for l in range(val_pred.shape[0]):
                val_pred[l]=val_pred[l][-6:]
            for l in range(test_pred.shape[0]):
                test_pred[l]=test_pred[l][-6:]
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)

        train_pred_final=np.array(encoder2.predict(train_pred_norm_auto))
        val_pred_final=np.array(encoder2.predict(val_pred_norm_auto))
        test_pred_final=np.array(encoder2.predict(test_pred_auto))

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,32),name='input_pred')

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=add([branch_0,input_pred])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,input_pred])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,input_pred])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

        model=Model(inputs=[input_data,input_pred],outputs=net)
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF02.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

        model.fit({'input_data':train_input,'input_pred':train_pred_final},y_train,validation_data=[[val_input,val_pred_final],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF02.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_final})
        pred=scaler3.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape2[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction02.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
    

In [29]:
print (sum(Final_smape2)/72)

14.177486495179359


# Case 3: Adding Expert Knowledge to layer 2

In [ ]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape3=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
    
        train = series_data[:-n_val]
        test = series_d[-(horizon+window_size):]
        val = series_data[-(2*n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            scaler1.fit(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1))
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            scaler2.fit(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1))
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            scaler3.fit(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1))
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        #val_input = np.append(x_val,val_pred_auto,axis=1)
        val_input = x_val
        #test_input = np.append(x_test,test_pred_auto,axis=1)
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        train_p = series_pred[:-n_val]
        val_p = series_pred[-(2*n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l]=train_pred[l][-6:]
            for l in range(val_pred.shape[0]):
                val_pred[l]=val_pred[l][-6:]
            for l in range(test_pred.shape[0]):
                test_pred[l]=test_pred[l][-6:]
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)

        train_pred_final=np.array(encoder2.predict(train_pred_norm_auto))
        val_pred_final=np.array(encoder2.predict(val_pred_norm_auto))
        test_pred_final=np.array(encoder2.predict(test_pred_auto))

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,32),name='input_pred')

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        #branch_0=add([branch_0,input_pred])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_1=add([branch_1,input_pred])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        #branch_2=add([branch_2,input_pred])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

        model=Model(inputs=[input_data,input_pred],outputs=net)
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF03.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

        model.fit({'input_data':train_input,'input_pred':train_pred_final},y_train,validation_data=[[val_input,val_pred_final],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF03.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_final})
        pred=scaler3.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape3[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction03.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
    

In [30]:
print (sum(Final_smape3)/72)

14.712916680754685


# Case 4: Adding Expert Knowledge to layer 3

In [ ]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape4=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
    
        train = series_data[:-n_val]
        test = series_d[-(horizon+window_size):]
        val = series_data[-(2*n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            scaler1.fit(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1))
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            scaler2.fit(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1))
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            scaler3.fit(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1))
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        #val_input = np.append(x_val,val_pred_auto,axis=1)
        val_input = x_val
        #test_input = np.append(x_test,test_pred_auto,axis=1)
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        train_p = series_pred[:-n_val]
        val_p = series_pred[-(2*n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l]=train_pred[l][-6:]
            for l in range(val_pred.shape[0]):
                val_pred[l]=val_pred[l][-6:]
            for l in range(test_pred.shape[0]):
                test_pred[l]=test_pred[l][-6:]
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)

        train_pred_final=np.array(encoder2.predict(train_pred_norm_auto))
        val_pred_final=np.array(encoder2.predict(val_pred_norm_auto))
        test_pred_final=np.array(encoder2.predict(test_pred_auto))

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,32),name='input_pred')

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        #branch_0=add([branch_0,input_pred])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        #branch_1=add([branch_1,input_pred])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_2=add([branch_2,input_pred])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

        model=Model(inputs=[input_data,input_pred],outputs=net)
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF04.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

        model.fit({'input_data':train_input,'input_pred':train_pred_final},y_train,validation_data=[[val_input,val_pred_final],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF04.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_final})
        pred=scaler3.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape4[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction04.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
    

In [31]:
print (sum(Final_smape4)/72)

17.894485371826107


# Case 5: Adding Expert Knowledge to All 3 layers

In [ ]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape5=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
    
        train = series_data[:-n_val]
        test = series_d[-(horizon+window_size):]
        val = series_data[-(2*n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            scaler1.fit(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1))
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            scaler2.fit(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1))
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            scaler3.fit(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1))
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]
############################################################################################################
############################################################################################################
############################################################################################################
        train_input = x_train
        #val_input = np.append(x_val,val_pred_auto,axis=1)
        val_input = x_val
        #test_input = np.append(x_test,test_pred_auto,axis=1)
        test_input = x_test
####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        train_p = series_pred[:-n_val]
        val_p = series_pred[-(2*n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l]=train_pred[l][-6:]
            for l in range(val_pred.shape[0]):
                val_pred[l]=val_pred[l][-6:]
            for l in range(test_pred.shape[0]):
                test_pred[l]=test_pred[l][-6:]
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)

        train_pred_final=np.array(encoder2.predict(train_pred_norm_auto))
        val_pred_final=np.array(encoder2.predict(val_pred_norm_auto))
        test_pred_final=np.array(encoder2.predict(test_pred_auto))

####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size,1),name='input_data')
        input_pred=Input(batch_shape=(None,32),name='input_pred')

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_0=add([branch_0,input_pred])
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_1=add([branch_1,input_pred])
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_2=add([branch_2,input_pred])
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

        model=Model(inputs=[input_data,input_pred],outputs=net)
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF05.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

        model.fit({'input_data':train_input,'input_pred':train_pred_final},y_train,validation_data=[[val_input,val_pred_final],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF05.h5')
        pred=model.predict({'input_data':test_input, 'input_pred':test_pred_final})
        pred=scaler3.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape5[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction05.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
    

In [32]:
print (sum(Final_smape5)/72)

16.26664535383255


# Case 6: Considering Expert Knowledge as input

In [ ]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape6=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
    
        train = series_data[:-n_val]
        test = series_d[-(horizon+window_size):]
        val = series_data[-(2*n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            scaler1.fit(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1))
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            scaler2.fit(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1))
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            scaler3.fit(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1))
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        train_p = series_pred[:-n_val]
        val_p = series_pred[-(2*n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l]=train_pred[l][-6:]
            for l in range(val_pred.shape[0]):
                val_pred[l]=val_pred[l][-6:]
            for l in range(test_pred.shape[0]):
                test_pred[l]=test_pred[l][-6:]
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        test_pred_auto=pd.DataFrame(test_pred_auto)

        train_pred_final=np.array(encoder2.predict(train_pred_norm_auto))
        val_pred_final=np.array(encoder2.predict(val_pred_norm_auto))
        test_pred_final=np.array(encoder2.predict(test_pred_auto))

        
        ############################################################################################################
        train_input = np.append(x_train,train_pred_final.reshape(train_pred_final.shape[0],train_pred_final.shape[1],1),axis=1)
        val_input = np.append(x_val,val_pred_final.reshape(val_pred_final.shape[0],val_pred_final.shape[1],1),axis=1)
        test_input = np.append(x_test,test_pred_final.reshape(test_pred_final.shape[0],test_pred_final.shape[1],1),axis=1)
        
        
####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size+32,1),name='input_data')
        #input_pred=Input(batch_shape=(None,32),name='input_pred')

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

        model=Model(inputs=[input_data],outputs=net)
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF06.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

        model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF06.h5')
        pred=model.predict({'input_data':test_input})
        pred=scaler3.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape6[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction06.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
    

In [33]:
print (sum(Final_smape6)/72)

13.35650816480022


# Case 7: Adding Expert Knowledge (with size 4) to input Data

In [ ]:
print(datetime.datetime.now())
smape1=0
smape2=0
mse1=0
data_length = data.shape[0]
with tqdm(total=data_length) as pbar:
    final_predictions = np.zeros([data_length,12])
    final_test = np.zeros([data_length,12])
    Final_smape7=np.zeros(data_length)
    for y in range(data_length):
        final_predictions = np.zeros([data_length,12])
        final_test = np.zeros([data_length,12])
        horizon = data.iloc[y].values[1]
        window_size=6
        current_row =np.asarray(data.loc[y][3:].dropna().values,dtype=float)
        rr = current_row.size
        rr = int(np.floor(rr*.25))
        series_d=current_row[rr:] # values after (rr)th position in current_row
#####################################################################
        series_data = series_d[:-horizon]
        series_length = series_data.size
        n_val = int(np.round(series_length*.2))
    
        train = series_data[:-n_val]
        test = series_d[-(horizon+window_size):]
        val = series_data[-(2*n_val+window_size):]

        train_sequence = make_input(train, window_size,horizon)
        val_sequence = make_input(val,window_size,horizon)
        test_sequence = nonov_make_input(test,window_size,horizon)

        train_sequence_norm = deepcopy(train_sequence)
        val_sequence_norm = deepcopy(val_sequence)
        test_sequence_norm = deepcopy(test_sequence)

        for j in range(train_sequence[0].shape[0]):
            scaler1 = MinMaxScaler()
            scaler1.fit(train_sequence[0][j])
            train_sequence_norm[0][j]= scaler1.transform(train_sequence[0][j])
            scaler1.fit(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1))
            train_sequence_norm[1][j]= scaler1.transform(train_sequence[1][j].reshape(train_sequence[1][j].shape[0],1)).reshape(train_sequence[1][j].shape[0])
    

        for j in range(val_sequence[0].shape[0]):
            scaler2 = MinMaxScaler()
            scaler2.fit(val_sequence[0][j])
            val_sequence_norm[0][j]= scaler2.transform(val_sequence[0][j])
            scaler2.fit(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1))
            val_sequence_norm[1][j]= scaler2.transform(val_sequence[1][j].reshape(val_sequence[1][j].shape[0],1)).reshape(val_sequence[1][j].shape[0])

        for j in range(test_sequence[0].shape[0]):
            scaler3 = MinMaxScaler()
            scaler3.fit(test_sequence[0][j])
            test_sequence_norm[0][j]= scaler3.transform(test_sequence[0][j])
            scaler3.fit(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1))
            test_sequence_norm[1][j]= scaler3.transform(test_sequence[1][j].reshape(test_sequence[1][j].shape[0],1)).reshape(test_sequence[1][j].shape[0])
    
        current_test=test_sequence[1][0]


        x_train = train_sequence_norm[0]
        y_train =train_sequence_norm[1]
        x_val = val_sequence_norm[0]
        y_val = val_sequence_norm[1]
        x_test = test_sequence_norm[0]
        y_test = test_sequence_norm[1]

####################################################################################
#######creating train, validation and test sets from Predictions####################
####################################################################################
        current_pred= np.asarray(predictions.iloc[y].dropna().values,dtype=float)
#####################################################################
        series_p=current_pred
        series_pred=series_p[:-horizon]
        train_p = series_pred[:-n_val]
        val_p = series_pred[-(2*n_val+window_size):]
        test_p = series_p[-(horizon+window_size):]

        train_pred1 = make_k_input(train_p,window_size,horizon)
        val_pred1 = make_k_input(val_p,window_size,horizon)
        test_pred1 = nonov_make_k_input(test_p,window_size,horizon)

        if horizon==12 :
            train_pred = np.zeros((train_pred1.shape[0], 6, train_pred1.shape[2]))
            val_pred = np.zeros((val_pred1.shape[0], 6, val_pred1.shape[2]))
            test_pred = np.zeros((test_pred1.shape[0], 6, test_pred1.shape[2]))

            for l in range(train_pred1.shape[0]):
                train_pred[l]=train_pred[l][-6:]
            for l in range(val_pred.shape[0]):
                val_pred[l]=val_pred[l][-6:]
            for l in range(test_pred.shape[0]):
                test_pred[l]=test_pred[l][-6:]
        else:
            train_pred=train_pred1
            val_pred=val_pred1
            test_pred=test_pred1
    
        train_pred_norm=np.zeros(train_pred.shape)
        val_pred_norm=np.zeros(val_pred.shape)
        test_pred_norm=np.zeros(test_pred.shape)
    
        for j in range(train_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(train_pred[j])
            train_pred_norm[j]= scaler.transform(train_pred[j])
        for j in range(val_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(val_pred[j])
            val_pred_norm[j]= scaler.transform(val_pred[j])
        for j in range(test_pred.shape[0]):
            scaler = MinMaxScaler()
            scaler.fit(test_pred[j])
            test_pred_norm[j]= scaler.transform(test_pred[j])
    
        train_pred_norm=train_pred_norm.reshape(train_pred_norm.shape[0],6)
        val_pred_norm=val_pred_norm.reshape(val_pred_norm.shape[0],6)
        test_pred_norm=test_pred_norm.reshape(test_pred_norm.shape[0],6)



##########################################################################################################################
        ncol = 6
        encoding_dim = 4
        input_dim = Input(shape = (ncol, ))

        input_dim = Input(shape = (ncol, ))
        #encoder = Model(inputs = input_dim, outputs = encoded2)
        encoded_input = Input(shape = (encoding_dim, ))
        encoder=load_model('MyModelCheckpoint/encoder_model.h5')
        train_pred_norm_auto=np.array(encoder.predict(train_pred_norm))
        val_pred_norm_auto=np.array(encoder.predict(val_pred_norm))
        test_pred_auto=np.array(encoder.predict(test_pred_norm))

        #train_pred_norm_auto=pd.DataFrame(train_pred_norm_auto)
        #val_pred_norm_auto=pd.DataFrame(val_pred_norm_auto)
        #test_pred_auto=pd.DataFrame(test_pred_auto)

        #train_pred_final=np.array(encoder2.predict(train_pred_norm_auto))
        #val_pred_final=np.array(encoder2.predict(val_pred_norm_auto))
        #test_pred_final=np.array(encoder2.predict(test_pred_auto))

        train_pred_final=train_pred_norm_auto
        val_pred_final=val_pred_norm_auto
        test_pred_final=test_pred_auto
        ############################################################################################################
        train_input = np.append(x_train,train_pred_final.reshape(train_pred_final.shape[0],train_pred_final.shape[1],1),axis=1)
        val_input = np.append(x_val,val_pred_final.reshape(val_pred_final.shape[0],val_pred_final.shape[1],1),axis=1)
        test_input = np.append(x_test,test_pred_final.reshape(test_pred_final.shape[0],test_pred_final.shape[1],1),axis=1)
        
        
####################################################################################
################       Neural Network Configuration         ########################
####################################################################################
        input_data= Input(batch_shape=(None,window_size+4,1),name='input_data')
        #input_pred=Input(batch_shape=(None,32),name='input_pred')

        branch_0 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(input_data)
        branch_1 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_0)
        branch_2 = Conv1D(32,3, strides=1, padding='same',activation='relu',kernel_initializer=glorot_uniform(1))(branch_1)
        branch_3=Flatten()(branch_2)
        net= Dense(horizon,name='dense_final',activity_regularizer=regularizers.l2(0.03))(branch_3)

        model=Model(inputs=[input_data],outputs=net)
        callback = ModelCheckpoint(filepath='MyModelCheckpoint/CIF07.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001))

        model.fit({'input_data':train_input},y_train,validation_data=[[val_input],y_val],callbacks=[callback],batch_size=8,shuffle=True, epochs=75,verbose=0)

        model.load_weights('MyModelCheckpoint/CIF07.h5')
        pred=model.predict({'input_data':test_input})
        pred=scaler3.inverse_transform(pred)#######################################################

        final_predictions[y,:horizon] = pred.reshape(horizon)
        final_test[y,:horizon]=test[-horizon:]

        AAA=smape(pred.reshape(horizon),current_test,horizon)
        smape1+=AAA
        smape2+=smape_nzr(pred.reshape(horizon),current_test)
        Final_smape7[y]=AAA
        mse1+=mse(pred.reshape(horizon),current_test)
        pbar.update(1)

np.savetxt('MyModelCheckpoint/prediction07.csv',final_predictions, fmt='%1.3f',delimiter=',')
print(datetime.datetime.now())
print('-----------------')
print("SMAPE1: (DeepEX approach)")
print(smape1/72)
print("SMAPE2 (NZR approach)")
print(smape2/72)
print("MSE1:")
print(mse1/72)
    

In [7]:
print (sum(Final_smape7)/72)

13.209504387588556
